In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

In [2]:
# First let's define some global variables.

In [3]:
background = None

accumulated_weight = 0.1

roi_top = 240
roi_bottom = 440
roi_right = 380
roi_left = 600

In [4]:
# We'll set up a function that updates a running average of the background values in a ROI.

In [5]:
def calc_accum_avg(frame, accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float')
        return None
    
    else:
        cv2.accumulateWeighted(frame, background, accumulated_weight)

In [6]:
# The next step is to use thresholding to grab the hand segment from the ROI

In [7]:
def segment(frame, threshold_min=25):
    
    # Calculate the absolute difference between the background and the passed frame
    diff = cv2.absdiff(background.astype('uint8'),frame)
    
    # Apply the threshold
    ret, thresholded = cv2.threshold(diff, threshold_min, 255, cv2.THRESH_BINARY)
    
    # We wanna grab the external contours from the image
    image, contours, hierarchy = cv2.findContours(thresholded.copy(),
                                                  cv2.RETR_EXTERNAL,
                                                  cv2.CHAIN_APPROX_SIMPLE)
    
    # This means that we didn't grab any contours
    if len(contours) == 0: 
        return None
    
    # Assuming the largest external contour in roi is the hand 
    else:       
        hand_segment = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment)

In [8]:
# Finger counting using Convex Hull

In [9]:
def count_fingers(thresholded, hand_segment, frame):
    
    # We create the convex hull for the hand
    conv_hull = cv2.convexHull(hand_segment)
    
    # Calculate the more extreme points
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    # Display those extreme points (for debugging)
    #cv2.circle(frame, (top[0]+roi_right,top[1]+roi_top), 5, (0,255,255), -1)
    #cv2.circle(frame, (bottom[0]+roi_right,bottom[1]+roi_top), 5, (0,255,255), -1)
    #cv2.circle(frame, (left[0]+roi_right,left[1]+roi_top), 5, (0,255,255), -1)
    #cv2.circle(frame, (right[0]+roi_right,right[1]+roi_top), 5, (0,255,255), -1)
    
    
    # Calculate the center of the hand
    center_x = (left[0] + right[0]) // 2
    center_y = (top[1] + bottom[1]) // 2
    
    # Display the center of the hand (for debugging)
    cv2.circle(frame, (center_x+roi_right,center_y+roi_top), 5, (0,255,255), -1)
    
    # Calculate the Euclidean distance between the center of the hand and the left, right, top and bottom points
    distance = pairwise.euclidean_distances([[center_x,center_y]], [np.array(left),np.array(right),np.array(top),np.array(bottom)])[0]
    
    # We are just interested in the distance to the further point
    max_distance = distance.max()
    
    # We'll create a circle of radius = 80% of the max distance. Those points touching the circle will be counted as fingers.
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)
    
    # Display that circle (for debugging)
    cv2.circle(frame, (center_x+roi_right,center_y+roi_top), radius, (0,255,255), 1) 
    
    # We create a new region of interest that we'll be using as a mask later
    circular_roi = np.zeros(shape=(200,220),dtype='uint8')
    
    # We draw the circle now on our new roi, the thickness will be our mask
    cv2.circle(circular_roi, (center_x,center_y), radius, 255, 10)
    
    # We apply the mask to the thresholded image
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    
    # Display the result of the mask (for debugging)
    cv2.imshow('Circular Mask', circular_roi)
    
    # We obtain the contours for the masked result
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # Count the number of points touching the circle
    count = 0
    
    # For every contour in that list of contours
    for cnt in contours:
        
        # Grab the bounding box of the contour
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        # We wanna make sure that the contour region is not at the very bottom of the hand
        # We don't want to count points belonging to the person forearm
        out_of_wrist = (center_y + (center_y * 0.25)) > (y+h)
        
        # We also wanna make sure that the number of points along the contour doesn't exceed 25% of the circumference of the circular roi
        # Otherwise we are basically counting points that are outside the hand itself
        limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        # If it pass the 2 conditions we conclude that is a finger
        if out_of_wrist and limit_points:
            
            count += 1
    
    return count

In [10]:
# Bringing it all together

In [11]:
# Grab live video
cam = cv2.VideoCapture(0)
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('Final Project.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))

num_frames = 0

while True:
    
    ret, frame = cam.read()
    
    frame_copy = frame.copy()
    
    # This is the region of interest where we will put our hand
    roi = frame[roi_top:roi_bottom, roi_right:roi_left]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    # We apply a little bit of blur
    gray = cv2.GaussianBlur(gray, (7,7), 0)
    
    # We will calculate the background for the first 60 frames (before we put our hand in the region of interest)
    if num_frames < 60:
        
        calc_accum_avg(gray, accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy, 'WAIT. GETTING BACKGROUND', (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Finger Count', frame_copy)
            
    # Now we are ready to enter the hand in the roi
    else:
        
        # Segments the hand region
        hand = segment(gray)
        
        # If we detect a hand
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            # Draw contours around real hand in live stream
            cv2.drawContours(frame_copy, [hand_segment+(roi_right,roi_top)], -1, (255,0,0), 2)
            
            # Count the fingers
            fingers = count_fingers(thresholded, hand_segment, frame_copy)
            
            # Display the count
            cv2.putText(frame_copy, str(fingers), (380,233), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
            
            # Let's also show the thresholded image
            cv2.imshow('Thresholded', thresholded)
            
    # Draw a rectangle of the region of interest
    cv2.rectangle(frame_copy, (roi_left,roi_top), (roi_right,roi_bottom), (0,0,255), 1)
    
    # Save video
    writer.write(frame_copy)
    
    # Show video
    cv2.imshow('Finger Count', frame_copy)
    
    # Frames counter
    num_frames += 1
    
    # Escape instance
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        

cam.release()
writer.release()
cv2.destroyAllWindows()
        
